In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

# Carga de datos
gtd_data = pd.read_csv("/kaggle/input/gtd")
print(gtd_data.info())

# ------------- LIMPIEZA PREVIA A LA SELECCIÓN -------------

# Convertimos a NaN los espacios en blanco:
gtd_data_limpieza = gtd_data.replace(["", " ", "Unknown", -9, -99], np.nan)
print(gtd_data_limpieza.info())

# Quitamos columnas con terminación _txt
columnas_txt = gtd_data_limpieza.columns[gtd_data_limpieza.columns.str.endswith('_txt')]
print(columnas_txt)

def quitar_terminacion_txt(columna_txt):
    return columna_txt.replace('_txt', '')

# Crear una función para asociar las filas únicas y llenar valores faltantes
def crear_asociacion_unica_y_llenar(gtd_data, variable1, variable2):
    asociacion_unica = {}
    gtd_data[variable2] = gtd_data[variable2].astype(str).replace(["", " "], np.nan)

    for i in range(len(gtd_data)):
        if pd.notna(gtd_data.loc[i, variable1]) and pd.notna(gtd_data.loc[i, variable2]):
            asociacion_unica[gtd_data.loc[i, variable1]] = gtd_data.loc[i, variable2]

    for i in range(len(gtd_data)):
        if pd.isna(gtd_data.loc[i, variable1]) and pd.notna(gtd_data.loc[i, variable2]):
            gtd_data.loc[i, variable1] = next((key for key, value in asociacion_unica.items() if value == gtd_data.loc[i, variable2]), np.nan)
    
    return gtd_data

# Recorrer las columnas que terminan en "_txt"
for columna in columnas_txt:
    col_sin_txt = quitar_terminacion_txt(columna)
    gtd_data_limpieza = crear_asociacion_unica_y_llenar(gtd_data_limpieza, col_sin_txt, columna)

# Eliminar columnas con terminación en _txt
gtd_data_limpieza.drop(columns=columnas_txt, inplace=True)

print(gtd_data_limpieza.info())

# -- Eliminación de columnas con más del 90% de valores perdidos: --

def eliminar_columnas_valores_perdidos(datos, umbral=90):
    porcentaje_perdido = datos.isna().mean() * 100
    columnas_a_eliminar = porcentaje_perdido[porcentaje_perdido > umbral].index
    return datos.drop(columns=columnas_a_eliminar)

gtd_data_limpieza = eliminar_columnas_valores_perdidos(gtd_data_limpieza, 90)
print(gtd_data_limpieza.info())

# Eliminamos columnas de forma manual
columnas_a_eliminar = ["location", "summary", "nwound", "propcomment", "addnotes", "scite2", "scite3", "related"]
gtd_data_limpieza.drop(columns=columnas_a_eliminar, inplace=True)
print(gtd_data_limpieza.info())

# ---- IGUALAMOS LA CANTIDAD DE INSTANCIAS DE success = 0 CON las = a 1 ---- 
gtd_data_s1 = gtd_data_limpieza[gtd_data_limpieza['success'] == 1]
gtd_data_s0 = gtd_data_limpieza[gtd_data_limpieza['success'] == 0]
occurrences_s0 = len(gtd_data_s0)
gtd_data_s1 = gtd_data_s1.sample(n=occurrences_s0, random_state=123)
muestra = pd.concat([gtd_data_s1, gtd_data_s0], ignore_index=True)

# ---- SELECCIÓN DE ATRIBUTOS CON CHI^2 -----
X = muestra.drop(columns='success')
y = muestra['success']

# Usando chi-squared para seleccionar las mejores características
chi2_selector = SelectKBest(chi2, k='all')
X_chi2 = chi2_selector.fit_transform(X.select_dtypes(include=[np.number]), y)

# Crear un DataFrame con las importancias
pesos = pd.DataFrame(chi2_selector.scores_, index=X.select_dtypes(include=[np.number]).columns, columns=['attr_importance'])
orden = pesos['attr_importance'].sort_values()
pesos = pesos[pesos['attr_importance'] > 0]

# Visualizar la importancia
plt.figure(figsize=(10, 6))
plt.barh(orden.index, orden.values)
plt.xlabel("Importancia")
plt.title("Importancia de Atributos - Chi²")
plt.show()

# Crea muestra con las variables que nos sirven
gtd_data_seleccion = muestra[pesos.index]
gtd_data_seleccion.insert(0, 'eventid', muestra['eventid'])
gtd_data_seleccion['success'] = muestra['success']

print(gtd_data_seleccion.info())

# ---- TRATAMIENTO DE VALORES PERDIDOS -----

# Reemplazando por media / moda / mediana
def mode2(x):
    return x.mode()[0]

def imputacion(data):
    for var in data.columns:
        if data[var].dtype == 'float64':
            data[var].fillna(data[var].mean(), inplace=True)
        elif data[var].dtype == 'object':
            no_empty = data[var].dropna().replace("", np.nan).dropna()
            m = mode2(no_empty)
            data[var].fillna(m, inplace=True)
            data[var].replace("", m, inplace=True)
        elif data[var].dtype == 'int64':
            data[var].fillna(data[var].median(), inplace=True)
    return data

gtd_data_sin_vp = imputacion(gtd_data_seleccion)
print(gtd_data_sin_vp.info())

# Reemplazando usando aprendizaje automático:
from missingpy import MissForest

# Usar un muestreo del 1%
porcentaje_muestreo = 0.01
tamano_muestra = int(len(muestra) * porcentaje_muestreo)
muestra = muestra.sample(n=tamano_muestra, random_state=1)
print(muestra.info())

# Imputar los valores perdidos usando MissForest
imputador = MissForest()
gtd_data_sin_vp2 = imputador.fit_transform(muestra.select_dtypes(include=[np.number]))
gtd_data_sin_vp2 = pd.DataFrame(gtd_data_sin_vp2, columns=muestra.select_dtypes(include=[np.number]).columns)

# ---- ELIMINACIÓN DE VALORES ATÍPICOS -----
columnas_con_nas = gtd_data_sin_vp.columns[gtd_data_sin_vp.isna().any()].tolist()
print(columnas_con_nas)

def quitar_atipicos(columna, threshold=1.5):
    iqr = np.percentile(columna, 75) - np.percentile(columna, 25)
    lower_limit = np.percentile(columna, 50) - threshold * iqr
    upper_limit = np.percentile(columna, 50) + threshold * iqr
    return columna.mask((columna < lower_limit) | (columna > upper_limit))

for nombre_columna in gtd_data_sin_vp.select_dtypes(include=[np.number]).columns:
    gtd_data_sin_vp[nombre_columna] = quitar_atipicos(gtd_data_sin_vp[nombre_columna])

print(gtd_data_sin_vp.head())
print(gtd_data_sin_vp.info())

hay_nas_en_data = gtd_data_sin_vp.isna().any().any()
print(hay_nas_en_data)

# ---- DISCRETIZACIÓN -----
def discretizar_por_rango(data):
    columnas_numericas = data.select_dtypes(include=[np.number]).columns
    for columna in columnas_numericas:
        print(f"Rango de {columna}: {data[columna].min()} - {data[columna].max()}")
        num_bins = int(np.ceil(np.log2(len(data[columna])) + 1))
        data[columna] = pd.cut(data[columna], bins=num_bins)
    return data

gtd_data_discretizado = discretizar_por_rango(gtd_data_sin_vp.copy())
print(gtd_data_discretizado.head())
